In [5]:
'''
03_create-paths.ipynb
Create a dataset of shortest paths between all destinations
'''

import pathlib
import os
import pandas as pd
import geopandas as gpd
import numpy as np
from shapely import wkt
from shapely.geometry import Point, Polygon, LineString
from shapely.ops import nearest_points, unary_union
import math
import json
import geojson
import h3
import networkit as nk
from osgeo import gdal

PATH_ROOT = os.path.join(pathlib.Path().absolute(), '../..' )
PATH_IN = PATH_ROOT + '/data/02_processed/'
PATH_OUT = PATH_ROOT + '/data/03_paths/'
os.makedirs(PATH_OUT) if not os.path.exists(PATH_OUT) else False

# constants
CONST_graph_detail_lvl = 8
CONST_speed_day_km = 40
CONST_slope_effect_multiplier = 2
CONST_distance_bridge = 10 / 111
CONST_river_value = 9999

# critical slope algorithm
CONST_critical_slope = 4

# for tobler application
CONST_elevation_coefficient = 1

def slope_coeff(slope):
    return 1 / (1 + ((abs(slope) * 100) / CONST_critical_slope) ** 2)

def calculate_path_time(value_from, value_to, dist):
    rise = (value_to - value_from) / 1000
    if value_from == CONST_river_value and value_to == CONST_river_value:
        return 10000
    else:
        slope = rise / dist
        slope_c = slope_coeff(slope)
        time = CONST_speed_day_km * abs(slope_c ** CONST_slope_effect_multiplier)

        el_dist = math.sqrt((rise ** 2) + (dist ** 2))
        path_time = (el_dist / time)

        #print(value_to, value_from, slope, time, el_dist, path_time)
        return path_time

print(calculate_path_time(CONST_river_value, CONST_river_value, 10))
print(calculate_path_time(50, 200, 40))
print(calculate_path_time(200, 50, 40))

hex_distance = 2 * h3.edge_length(CONST_graph_detail_lvl)
print('hex_distance', hex_distance)

# load datasets

destinations = gpd.read_file(PATH_IN + 'destinations.geojson')

rivers_df = gpd.read_file(PATH_IN + 'rivers.geojson') 
rivers = unary_union(
    [river['geometry'] for ri, river in rivers_df.iterrows()]
)

bridges_df = gpd.read_file(PATH_IN + 'bridges.geojson')
bridges = unary_union(
    [bridge['geometry'].buffer(hex_distance / 222) for bi, bridge in bridges_df.iterrows()]
)

bbox = gpd.read_file(PATH_IN + 'bbox.geojson') 
bb_xy = bbox.total_bounds

bounds_json = dict(
    geojson.Polygon(
        [[
            [bb_xy[0], bb_xy[1]],
            [bb_xy[0], bb_xy[3]],
            [bb_xy[2], bb_xy[3]],
            [bb_xy[2], bb_xy[1]],
            [bb_xy[0], bb_xy[1]],
        ]]
    ))


10000
1.017662527983812
1.017662527983812
hex_distance 0.922709368


In [6]:
# load elevation data

elevation = gdal.Open(PATH_IN + 'elevation.tif')
band = elevation.GetRasterBand(1)
cols = elevation.RasterXSize
rows = elevation.RasterYSize

transform = elevation.GetGeoTransform()
xOrigin = transform[0]
yOrigin = transform[3]
pixelWidth = transform[1]
pixelHeight = -transform[5]

elevation_data = band.ReadAsArray(0, 0, cols, rows)

# check whether there is river or a bridge
def path_value_point (hex_center):
    #return elevation_point(hex_center)
    #on_river = hex.intersects(rivers)
    on_river = hex_center.distance(rivers) < hex_distance / 200
    if on_river: 
        on_bridge = hex_center.distance(bridges) < hex_distance / 200
        if not on_bridge:
            value = CONST_river_value
        else:
            value = elevation_point(hex_center)
    else:
        value = elevation_point(hex_center)
    return value

# get elevation value for the given geographical point
def elevation_point (point):
    row = int((yOrigin - point.y ) / pixelHeight)
    col = int((point.x - xOrigin) / pixelWidth)
    
    if elevation_data.shape[0] > row and elevation_data.shape[1] > col:
        el_value = int(elevation_data[row][col])
        if el_value <= 0: # sea
            return CONST_river_value
        else:
            return el_value
    else:
        return CONST_river_value

In [8]:
# construct hexes dataframe (could take a while)

hex_ids = list(h3.polyfill(bounds_json, CONST_graph_detail_lvl))

hexes_df = pd.DataFrame(hex_ids, columns=['id'])
hexes_df['center'] = hexes_df.apply(
    lambda x: Point(h3.h3_to_geo(x['id'])),
    axis = 1
)
hexes_df["value"] = hexes_df.apply(
    lambda x: path_value_point(x['center']),
    axis = 1
)
hexes_df.set_index('id', inplace=True)

# find neighboring hexes
hexes_df['neighbors'] = hexes_df.apply(
    lambda x: \
        [
            h3.get_destination_h3_index_from_unidirectional_edge(n) 
            for n in h3.get_h3_unidirectional_edges_from_hexagon(x.name)\
        ],
    axis = 1
)

# save hexes_df
hexes_df.to_csv(PATH_OUT + 'hexes.csv')

In [9]:
hexes_df

,center,value,neighbors
id,,,
885271c0e1fffff,POINT (19.18034918736021 49.29193982631583),686,"[885271c0e3fffff, 885271c0edfffff, 885271c0e5f..."
8852c114dbfffff,POINT (12.25758501333742 46.98703765487186),2796,"[8852c114d9fffff, 8852c114d1fffff, 8852c1ab2df..."
885245a52dfffff,POINT (18.74251093501929 51.95509928866501),99,"[885245ae5bfffff, 885245a525fffff, 885245a567f..."
885274223bfffff,POINT (20.03858800000788 48.65631862353475),535,"[88527422edfffff, 8852742233fffff, 8852742231f..."
8863aac6e9fffff,POINT (12.16024164471779 53.80246959670126),4,"[8863aac6e1fffff, 8863aac617fffff, 8863aac6edf..."
...,...,...,...
88526db323fffff,POINT (20.26036822277168 54.2016394179755),103,"[88526db321fffff, 88526db33dfffff, 88526db335f..."
88525b10bbfffff,POINT (14.31783529443134 50.84076753689852),335,"[88525b1097fffff, 88525b1095fffff, 88525b10b9f..."
88524a92e3fffff,POINT (16.29928930334786 52.2634639543156),107,"[88524a92a9fffff, 88524a928dfffff, 88524a9285f..."


In [10]:
# create weighted non-directional graph for all combinations of neighboring hexes

from ast import literal_eval

hexes_df = gpd.read_file(
    PATH_OUT + 'hexes.csv'
)
hexes_df['center'] = hexes_df['center'].apply(wkt.loads)
hexes_df['neighbors'] = hexes_df['neighbors'].apply(literal_eval)
hexes_df["value"] = pd.to_numeric(hexes_df["value"])
hexes_df.set_index('id', inplace=True)

g = nk.Graph(directed=False, weighted=True)

hexes_df['node'] = hexes_df.apply(
    lambda x: g.addNode(),
    axis = 1
)

# node_id -> hex_id shorthand dictionary
node_to_hex = {node_id: hex_id for (hex_id, node_id) in hexes_df[['node']].itertuples()}

for (hex_from_id, value_from, node_from, neighbors) in hexes_df[['value', 'node', 'neighbors']].itertuples():
    for hex_to_id in neighbors:
        if hex_to_id in hexes_df.index:
            node_to = hexes_df.at[hex_to_id, 'node']
            value_to = hexes_df.at[hex_to_id, 'value']
            g.addEdge(node_from, node_to, calculate_path_time(value_from, value_to, hex_distance))

g.indexEdges()

In [11]:
# run dijkstra algorithm for all destinations

destinations['hex_id'] = destinations.apply(
    lambda x: h3.geo_to_h3(x['geometry'].x, x['geometry'].y, CONST_graph_detail_lvl),
    axis=1
)
destinations['node_id'] = destinations.apply(
    lambda x: hexes_df.at[x['hex_id'], 'node'],
    axis=1
)

In [12]:
# calculate hex id for each destination

paths = []

for (id_from, name_from, hex_from, node_from) in destinations[['name', 'hex_id', 'node_id']].itertuples():

    print('finding paths for destinations {}/{} ({}%)'.format(id_from, len(destinations), int(id_from / len(destinations) * 100)))

    dij = nk.distance.Dijkstra(g, node_from, True, False)
    dij.run()
    for (id_to, name_to, hex_to, node_to) in destinations[['name', 'hex_id', 'node_id']].itertuples():
         

        dist = dij.distance(node_to)
        path = dij.getPath(node_to)
        path_hexes = LineString([hexes_df.at[node_to_hex[i], 'center'] for i in path])

        paths.append(
            {
                "from": name_from,
                "to": name_to,
                "dist": dist,
                "geometry": path_hexes
            }
        )


finding paths for destinations 0/127 (0%)
finding paths for destinations 1/127 (0%)
finding paths for destinations 2/127 (1%)
finding paths for destinations 3/127 (2%)
finding paths for destinations 4/127 (3%)
finding paths for destinations 5/127 (3%)
finding paths for destinations 6/127 (4%)
finding paths for destinations 7/127 (5%)
finding paths for destinations 8/127 (6%)
finding paths for destinations 9/127 (7%)
finding paths for destinations 10/127 (7%)
finding paths for destinations 11/127 (8%)
finding paths for destinations 12/127 (9%)
finding paths for destinations 13/127 (10%)
finding paths for destinations 14/127 (11%)
finding paths for destinations 15/127 (11%)
finding paths for destinations 16/127 (12%)
finding paths for destinations 17/127 (13%)
finding paths for destinations 18/127 (14%)
finding paths for destinations 19/127 (14%)
finding paths for destinations 20/127 (15%)
finding paths for destinations 21/127 (16%)
finding paths for destinations 22/127 (17%)
finding pat

In [13]:
# export paths

# load paths
paths_df = gpd.GeoDataFrame(paths, crs="epsg:4326")

# simplify geometries
paths_df['geometry'] = paths_df.apply(
      lambda x: wkt.loads(
          wkt.dumps(
              x['geometry'].simplify(0.01, preserve_topology=True), 
              rounding_precision=3
            )
        ),
      axis=1
  )
#paths_df.to_file(PATH_OUT + 'paths.shp', driver="ESRI Shapefile", encoding="utf-8")
paths_df.to_file(PATH_OUT + 'paths.geojson', driver="GeoJSON")

In [14]:
# create a destination metrix table

paths_text = "".join([line.strip() for line in open(PATH_OUT + 'paths.geojson')])
paths_dict = json.loads(paths_text)

origins = {}
for fi, feat in enumerate(paths_dict['features']):
    orig = feat['properties']['from'] 
    origins[orig] = {'origin': orig}

for fi, feat in enumerate(paths_dict['features']):
    orig = feat['properties']['from']
    dest = feat['properties']['to']
    dist = feat['properties']['dist']
    origins[orig][dest] = dist

dist_df = pd.DataFrame(origins.values())


dist_df.set_index('origin', inplace=True)
dist_df.to_csv(PATH_OUT + 'dist_m.csv')

print(dist_df.at['Brno', 'Praha'])
print(dist_df.at['Praha', 'Wien'])


9.314863577278595
11.569084678024318
